# Simulation of Belvarafenib and Cobimetinib inhibition in BRAF<sup>V600E</sup> melanoma cells using the MARM2 model

Here you can simulate the respose of BRAF<sup>V600E</sup> cells under different doses of RAF (belvarafenib) and MEK inhibitors (cobimetinib). 

**Note**: this code performs the simulation for steady state responses under different inhibitor conditions. Use the Jupyter Notebook *Simulate_Belva_Cobi_Traj_MARM2_BRAF_V600E.ipynb* to generate simulation results for time course trajectories. 

## Import of libraries
Importing libraries necessary to run MARM2 model simulations.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import copy as cp
import itertools
import os
from pathlib import Path
import time
start_time = time.time()

Changes to main file directory. This works provided the "main_dir" has the correct directory name and the cwd starts within the main file directory. This might require tweaking under some high performance computing setups.

In [2]:
main_dir = "MARM2_Andrew_8_24"
for i in Path().resolve().parents:
    if i.parts[-1] == main_dir:
        os.chdir(i)

Importing the MARM2 PySB model and the simulator.  

In [3]:
from pysb.simulator import ScipyOdeSimulator
from pysb.core import as_complex_pattern
from pysb.bng import generate_equations

from scripts.models.MARM2_BRAF_V600E_No_RTK import model
from scripts.functions.pysb_helper_functions import equilibrate, get_species_index

## Generate model equations
PySB runs BioNetGen to generate the reaction network

In [4]:
generate_equations(model)

## Parameter set preparation

Loads and prepares the parameter sets described in Fr&ouml;hlich et al [1]. 

In [5]:
param_sets = pd.read_csv(Path('data/Model_parameters/RTKERK_pRAF_EGF_EGFR_MEKi_PRAFi_RAFi.csv'), index_col=0)
# finds the parameters of the .csv file that correspond to Cobimetinib and Vemurafenib and maps them to MEKi and RAFi (respectively)
rename_dict = {}
for i in param_sets.columns:
    if "Cobimetinib" in i or "Vemurafenib" in i:
        rename_dict[i] = i.replace("Cobimetinib","MEKi").replace("Vemurafenib","RAFi")
param_sets = param_sets.rename(columns = rename_dict)

# finds the parameters which are stored in .csv file but not in the model and removes them
csv_spec_params = set(param_sets.columns)-(set(param_sets.columns)&set([i.name for i in model.parameters]))
param_sets = param_sets.drop(csv_spec_params, axis=1)

# Removes preference against second inhibitor binding to model type 2 pan RAF inhibitor (Belvarafenib)
param_sets["ep_RAF_RAF_mod_RAFi_double_ddG"] = 0

# specifies which of the 50 best-fit parameter sets to use for the simulation. Set 0 is the best fit and 49 the worst.
param_set_index = 13
params = param_sets.iloc[param_set_index].to_dict()

params['RAFi_0'] = 0.0
params['MEKi_0'] = 0.0

## Initial equilibrium
First we run the model from its baseline initial conditions until equilibrium is reached. For example protein synthesis/degradation, phosphorylation/dephosphorylation, and drug binding/unbinding all need to reach steady state to match the state of the cells in the experimental setup. There may be some time without visible progress as behind the scenes PySB runs BioNetGen to generate the reaction network and Cython to compile the resulting differential equations into efficient executable code.

In [6]:
sim = ScipyOdeSimulator(model,param_values=params) 
df_eq = equilibrate(sim, None)

    at t=100   ... 875/875 species converged


Now that the model has been simulated once and the actual molecular species have been enumerated, we can find the exact species numbers for the inhibitors. These are needed so that their concentrations can be overridden in the model state for subsequent simulations.

In [7]:
RAFi_index = get_species_index(model, model.monomers.RAFi(raf=None)**model.compartments.CP)
MEKi_index = get_species_index(model, model.monomers.MEKi(mek=None)**model.compartments.CP)

## Inhibitor treatment

We take the final state of the equilibration simulation and use it as the initial state of this new simulation, overriding the RAFi and MEKi concentrations with pre-specified values.

In [8]:
#set the dilution range for the RAF inhibitor, which is x axis
RAFi_dil=np.logspace(-2.25,.5, 9); #uM
RAFi_dil = np.concatenate(([0],RAFi_dil))
#set the dilution range for the MEK inhibitor, which is y axis
MEKi_dil=np.logspace(-2.75,0, 9); #uM
MEKi_dil=np.concatenate(([0],MEKi_dil))
print(RAFi_dil)
print(MEKi_dil)

[0.         0.00562341 0.01240938 0.0273842  0.06042964 0.13335214
 0.29427272 0.64938163 1.43301257 3.16227766]
[0.         0.00177828 0.00392419 0.00865964 0.01910953 0.04216965
 0.0930572  0.2053525  0.45315836 1.        ]


In [9]:
initials_pre = df_eq.iloc[-1, :len(model.species)].copy()
# specify observables to save
plt_obs=['Active_RAF_percent','pMEK', 'pMEK_percent', 'pERK', 'pERK_percent','Bound_CRAF_percent','Bound_RAF_percent'];
dr_df = pd.DataFrame(columns = ["RAFi_0_uM","MEKi_0_uM"]+plt_obs)
dose_ind = list(itertools.product(*[RAFi_dil,MEKi_dil]))
dr_df["RAFi_0_uM"] = [i[0] for i in dose_ind]
dr_df["MEKi_0_uM"] = [i[1] for i in dose_ind]
dr_df = dr_df.set_index(["RAFi_0_uM","MEKi_0_uM"])

# simulate each dose sequentially
results = []
for i in range(len(dose_ind)):
    print(np.round(100*(i+1)/len(dose_ind),1),end="% ")
    initials_pre[RAFi_index] = dose_ind[i][0]
    initials_pre[MEKi_index] = dose_ind[i][1]
    res = equilibrate(sim, initials_pre.to_list(),verbose=False).iloc[-1]
    dr_df.loc[dose_ind[i]] = res[plt_obs]
    
dr_df.to_csv(Path("data/Steady_state_dose_responses/Belva_Cobi_SS_BRAF_V600E"))

1.0% 2.0% 3.0% 4.0% 5.0% 6.0% 7.0% 8.0% 9.0% 10.0% 11.0% 12.0% 13.0% 14.0% 15.0% 16.0% 17.0% 18.0% 19.0% 20.0% 21.0% 22.0% 23.0% 24.0% 25.0% 26.0% 27.0% 28.0% 29.0% 30.0% 31.0% 32.0% 33.0% 34.0% 35.0% 36.0% 37.0% 38.0% 39.0% 40.0% 41.0% 42.0% 43.0% 44.0% 45.0% 46.0% 47.0% 48.0% 49.0% 50.0% 51.0% 52.0% 53.0% 54.0% 55.0% 56.0% 57.0% 58.0% 59.0% 60.0% 61.0% 62.0% 63.0% 64.0% 65.0% 66.0% 67.0% 68.0% 69.0% 70.0% 71.0% 72.0% 73.0% 74.0% 75.0% 76.0% 77.0% 78.0% 79.0% 80.0% 81.0% 82.0% 83.0% 84.0% 85.0% 86.0% 87.0% 88.0% 89.0% 90.0% 91.0% 92.0% 93.0% 94.0% 95.0% 96.0% 97.0% 98.0% 99.0% 100.0% 

In case the previous simulation was run to steady state, we want to retain only the first t_pretrt hours of pre-treatment plus the state at final equilibrium. So we cut the time series down using a Pandas slice operation and adjust the remaining time values to begin at -pre_time_max.

## References

[1] Fröhlich, F., Gerosa, L., Muhlich, J., & Sorger, P. K. (2023). Mechanistic model of MAPK signaling reveals how allostery and rewiring contribute to drug resistance. Molecular Systems Biology, 19(2), e10988. doi:10.15252/msb.202210988

# Session Info and Runtime

In [10]:
print("Elapsed Time = "+str(time.time() - start_time) + " seconds")
!conda env export --name quant_bio

Elapsed Time = 408.47760796546936 seconds
name: quant_bio
channels:
  - conda-forge
  - alubbock
  - defaults
dependencies:
  - anyio=4.2.0=py311hca03da5_0
  - appnope=0.1.2=py311hca03da5_1001
  - argon2-cffi=21.3.0=pyhd3eb1b0_0
  - argon2-cffi-bindings=21.2.0=py311h80987f9_0
  - asttokens=2.0.5=pyhd3eb1b0_0
  - async-lru=2.0.4=py311hca03da5_0
  - attrs=23.1.0=py311hca03da5_0
  - babel=2.11.0=py311hca03da5_0
  - beautifulsoup4=4.12.2=py311hca03da5_0
  - bionetgen=2.9.1=0
  - blas=1.0=openblas
  - bleach=4.1.0=pyhd3eb1b0_0
  - bottleneck=1.3.7=py311hb9f6ed7_0
  - brotli=1.0.9=h1a28f6b_7
  - brotli-bin=1.0.9=h1a28f6b_7
  - brotli-python=1.0.9=py311h313beb8_7
  - bzip2=1.0.8=h80987f9_5
  - ca-certificates=2024.2.2=hf0a4a13_0
  - certifi=2024.2.2=py311hca03da5_0
  - cffi=1.16.0=py311h80987f9_0
  - charset-normalizer=2.0.4=pyhd3eb1b0_0
  - comm=0.2.1=py311hca03da5_0
  - contourpy=1.2.0=py311h48ca7d4_0
  - cycler=0.11.0=pyhd3eb1b0_0
  - cython=3.0.8=py311h80987f9_0
  - debugpy=1.6.7=py311h31